# Using the search-API for the Norwegian Central Coordinating Register for Legal Entities


In [ ]:
!pip install requests

In [1]:
import requests

## Example: Get organisations with a certain activity type (NACE-code)


Let's say we want a list of organisation of the activity type of public libraries. There are two NACE-codes, 91.011 and 91.012 that are relevant. The search-API offers a [large set of properties](https://data.brreg.no/enhetsregisteret/api/docs/index.html#enheter-sok-detaljer) that can be used to specify the search, as query parameters to the API. In our case, we use the parameter naeringskode which takes a list of NACE-codes. In addition to the list of NACE-codes, we specify that we want the first page (i.e. 0), as well as the size of each page. To make the output easier to read we choose a low number, 2.

In [2]:
query_parametre = { 'naeringskode': [91.011,91.012], 'page': 0, 'size': 2 }

Now we are ready to run the request and save the result in the variable req:

In [3]:
req = requests.get('https://data.brreg.no/enhetsregisteret/api/enheter', params=query_parametre)

As we have requested a small size, we could show the result immediatly, but often it is OK to check if the request was successful, and look for status codes of 2xx:

In [4]:
req.status_code

200

If the status_code is 200, we can use the text-attribute to see the results. But since we are using an API that serves JSON, the formatting is better if we use the json-method:

In [16]:
req.json()

{'_embedded': {'enheter': [{'organisasjonsnummer': '920989330',
    'navn': 'F.DELBANCO GMBH & CO.KG',
    'organisasjonsform': {'kode': 'NUF',
     'beskrivelse': 'Norskregistrert utenlandsk foretak',
     '_links': {'self': {'href': 'https://data.brreg.no/enhetsregisteret/api/organisasjonsformer/NUF'}}},
    'registreringsdatoEnhetsregisteret': '2018-07-13',
    'registrertIMvaregisteret': False,
    'naeringskode1': {'beskrivelse': 'Drift av fag- og forskningsbiblioteker',
     'kode': '91.012'},
    'antallAnsatte': 0,
    'forretningsadresse': {'land': 'Tyskland',
     'landkode': 'DE',
     'poststed': '21339 Lüneburg',
     'adresse': ['Bessemerstrasse 3']},
    'registrertIForetaksregisteret': False,
    'registrertIStiftelsesregisteret': False,
    'registrertIFrivillighetsregisteret': False,
    'konkurs': False,
    'underAvvikling': False,
    'underTvangsavviklingEllerTvangsopplosning': False,
    'maalform': 'Bokmål',
    '_links': {'self': {'href': 'https://data.brreg.no

In [ ]:
### Storing the relevant data in a separate variable
Let's save the relevant part of the first response to a variable results. The relevant part is situated in a list under  _embedded > enheter. First we create the variable result, then we add the relevant content.

In [1]:
result = []
result += req.json()['_embedded']['enheter']

In [31]:
result

[{'organisasjonsnummer': '920989330',
  'navn': 'F.DELBANCO GMBH & CO.KG',
  'organisasjonsform': {'kode': 'NUF',
   'beskrivelse': 'Norskregistrert utenlandsk foretak',
   '_links': {'self': {'href': 'https://data.brreg.no/enhetsregisteret/api/organisasjonsformer/NUF'}}},
  'registreringsdatoEnhetsregisteret': '2018-07-13',
  'registrertIMvaregisteret': False,
  'naeringskode1': {'beskrivelse': 'Drift av fag- og forskningsbiblioteker',
   'kode': '91.012'},
  'antallAnsatte': 0,
  'forretningsadresse': {'land': 'Tyskland',
   'landkode': 'DE',
   'poststed': '21339 Lüneburg',
   'adresse': ['Bessemerstrasse 3']},
  'registrertIForetaksregisteret': False,
  'registrertIStiftelsesregisteret': False,
  'registrertIFrivillighetsregisteret': False,
  'konkurs': False,
  'underAvvikling': False,
  'underTvangsavviklingEllerTvangsopplosning': False,
  'maalform': 'Bokmål',
  '_links': {'self': {'href': 'https://data.brreg.no/enhetsregisteret/api/enheter/920989330'}}},
 {'organisasjonsnummer'

### Accessing the full result set with "page" and "size"
Based on the parameters it is likely that your result only contains a limited amount of the total result. The last element in the JSON-response, is information about how many elements, and how many pages with results, we have. We can focus on this element by treating it as a dictionary, and use 'page' as the key:

In [18]:
req.json()['page']

{'size': 2, 'totalElements': 21, 'totalPages': 11, 'number': 0}

To move forward in the list of pages, we can just update the relevant attribute in the variable query_parametre:


In [42]:
query_parametre['page'] += 1

Then we can re-run the request.


In [43]:
req = requests.get('https://data.brreg.no/enhetsregisteret/api/enheter', params=query_parametre)

Let's check if we got page two (which has number 1 ...):

In [44]:
req.json()['page']

{'size': 2, 'totalElements': 21, 'totalPages': 11, 'number': 2}

Now we can update our variable result with the new data by repeating the code that copied the relevant data to our variable result:

In [45]:
result += req.json()['_embedded']['enheter']

If our calculations are correct, the variable result should now be a list with four elements, each representing an organisation:

In [35]:
len(result)

4

Let's get a summary of the identifier, name and type of these elements. In Python this can be done with a for-loop, iterating through each element and extracting the relevant information. It can also be written as a list-comprehension as follows:

In [46]:
[(e['organisasjonsnummer'], e['navn'], e['organisasjonsform']['kode']) for e in result]

[('920989330', 'F.DELBANCO GMBH & CO.KG', 'NUF'),
 ('984456573', 'STIFTELSEN DE ANKERSKE SAMLINGER', 'STI'),
 ('984456573', 'STIFTELSEN DE ANKERSKE SAMLINGER', 'STI'),
 ('916087330', "SIGMUND HALDÅS' SAMLINGER", 'FLI'),
 ('982132541', 'KARASJOGA GIELDA / KARASJOK KOMMUNE BOKBUSSEN', 'ORGL'),
 ('986418989', 'HEMSEDAL HISTORIELAG', 'FLI')]

There is a risk that in the time between accessing the different pages, the content changes so that the same organisation listed in the bottom of one page becomes the first on the next page, or something similar, as the result-list is not kept static after the first request. So when using the results, it might be a good idea to remove duplicates.

### Automatically fetching all pages
Manually fetching each page is a bit tedious, even though in our example there is no more than ten pages. In our example there are not more than 21 elements in total, and if we increased size parameter in the request we would be able to get all elements in one response. Alternativly you can loop through the pages, and add the elements to your variable result.

*Warning*

There is an *important limitation* in the search-API that limits how many organisations it is possible to get access to using the search-API demonstrated here: If the sum of (page+1)*size > 10 000, the API returns and error (HTTP 400, "Bad request"). In our example it is not relevant, but if you get a response where the first page reports a "totalElements" of 10 000 or more, it will not be possible to get to more than the first 10 000 elements. Instead, you should use the functionality to download the full dataset. See [REF] for how to do that.

*Alternative 1*, changing the size of each page to a high enough number. In our case 50 is more than enough. Don't forget to make sure you ask for the first page, in case you have set the page-parameter earlier:

In [48]:
result_full = []
query_parametre = { 'naeringskode': [91.011,91.012], 'page': 0, 'size': 50 }
req = requests.get('https://data.brreg.no/enhetsregisteret/api/enheter', params=query_parametre)

In [49]:
result_full += req.json()['_embedded']['enheter']

This should result in the same length of the variable list result_full as reported as "totalElements".

In [58]:
req.json()['page']['totalElements'] == len(result_full)

True

*Alternative 2*, looping through all the pages, adding the result to a variable list

In [56]:
result_full_through_looping = []
query_parametre = { 'naeringskode': [91.011,91.012], 'page': 0, 'size': 2 }
url = 'https://data.brreg.no/enhetsregisteret/api/enheter'

req = requests.get(url, params=query_parametre) # Only to know size of result

total_pages = req.json()['page']['totalPages']

while query_parametre['page'] < total_pages:
    req = requests.get(url, params=query_parametre)
    result_full_through_looping += req.json()['_embedded']['enheter']

    # prepare for next iteration, updating the page-parameter in the request
    query_parametre['page'] += 1



In [59]:
req.json()['page']['totalElements'] == len(result_full_through_looping)

True

In [73]:
df = pd.read_json('er.json', compression='gzip' )

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037749 entries, 0 to 1037748
Data columns (total 43 columns):
 #   Column                                        Non-Null Count    Dtype  
---  ------                                        --------------    -----  
 0   Organisasjonsnummer                           1037749 non-null  int64  
 1   Navn                                          1037749 non-null  object 
 2   Organisasjonsform.kode                        1037749 non-null  object 
 3   Organisasjonsform.beskrivelse                 1037749 non-null  object 
 4   Næringskode 1                                 982062 non-null   float64
 5   Næringskode 1.beskrivelse                     982062 non-null   object 
 6   Næringskode 2                                 38573 non-null    float64
 7   Næringskode 2.beskrivelse                     38573 non-null    object 
 8   Næringskode 3                                 1562 non-null     float64
 9   Næringskode 3.beskrivelse          

In [20]:
df_sample = df.sample(10000)

In [21]:
len(df_sample)

10000

In [22]:
df_sample.describe()

,Organisasjonsnummer,Næringskode 1,Næringskode 2,Næringskode 3,Hjelpeenhetskode,Antall ansatte,Postadresse.postnummer,Postadresse.kommunenummer,Forretningsadresse.postnummer,Forretningsadresse.kommunenummer,Institusjonell sektorkode,Siste innsendte årsregnskap,Overordnet enhet i offentlig sektor
count,1.000000e+04,9447.000000,383.000000,20.000000,196.000000,10000.0000,1892.000000,1892.000000,9461.000000,9461.000000,9153.000000,3360.000000,2.100000e+01
mean,9.434304e+08,59.785001,42.585413,30.136350,65.342204,2.7470,3796.597252,2774.402748,3902.312335,2916.015537,5575.314105,2019.007738,9.470322e+08
std,4.379487e+07,29.479587,32.339305,33.282255,16.788348,41.7555,2851.340814,1739.036626,2616.633611,1646.625832,2859.231711,1.147087,3.641421e+07
min,8.115488e+08,0.000000,0.000000,1.110000,0.000000,0.0000,28.000000,301.000000,26.000000,301.000000,1120.000000,2002.000000,8.649660e+08
25%,9.184943e+08,43.341000,2.200000,2.052500,68.209000,0.0000,1337.000000,1103.000000,1593.000000,1506.000000,2100.000000,2019.000000,9.396077e+08
50%,9.260621e+08,68.209000,68.209000,5.155500,70.100000,0.0000,3455.500000,3024.000000,3619.000000,3030.000000,7000.000000,2019.000000,9.594123e+08
75%,9.854556e+08,86.211000,70.100000,54.821250,70.100000,0.0000,5863.000000,4224.250000,5690.000000,4225.000000,8200.000000,2019.000000,9.649636e+08
max,9.996668e+08,97.000000,96.010000,81.101000,82.990000,2350.0000,9991.000000,5444.000000,9990.000000,5444.000000,8300.000000,2020.000000,9.910121e+08


In [87]:
df_sample['organisasjonsnummer'] = df_sample['organisasjonsnummer'].astype('string')

In [89]:
df_sample['overordnetEnhet'] = df_sample['overordnetEnhet'].astype('string')

In [91]:
df.columns

Index(['organisasjonsnummer', 'navn', 'organisasjonsform',
       'registreringsdatoEnhetsregisteret', 'registrertIMvaregisteret',
       'naeringskode1', 'antallAnsatte', 'forretningsadresse',
       'institusjonellSektorkode', 'registrertIForetaksregisteret',
       'registrertIStiftelsesregisteret', 'registrertIFrivillighetsregisteret',
       'konkurs', 'underAvvikling',
       'underTvangsavviklingEllerTvangsopplosning', 'maalform', 'links',
       'stiftelsesdato', 'postadresse', 'naeringskode2', 'hjemmeside',
       'sisteInnsendteAarsregnskap', 'frivilligMvaRegistrertBeskrivelser',
       'naeringskode3', 'overordnetEnhet'],
      dtype='object')

In [28]:
df['Stiftelsesdato']

0                 NaN
1                 NaN
2          1990-01-30
3                 NaN
4                 NaN
              ...    
1037744    2021-04-20
1037745           NaN
1037746           NaN
1037747    2021-03-26
1037748    2021-02-05
Name: Stiftelsesdato, Length: 1037749, dtype: object

In [44]:
pd.to_datetime(df['Stiftelsesdato'], errors='coerce', infer_datetime_format=True) # uten errors='coerce' får jeg feilmelding. infer_datetime_format er for å spare tid på å forsøke ISO8601

0                NaT
1                NaT
2         1990-01-30
3                NaT
4                NaT
             ...    
1037744   2021-04-20
1037745          NaT
1037746          NaT
1037747   2021-03-26
1037748   2021-02-05
Name: Stiftelsesdato, Length: 1037749, dtype: datetime64[ns]

In [41]:
df.loc[pd.notnull(df['Stiftelsesdato']), ['Organisasjonsform.kode', 'Stiftelsesdato']].sort_values('Stiftelsesdato')

,Organisasjonsform.kode,Stiftelsesdato
948075,STI,1277-09-13
798696,STI,1538-12-31
46181,ANNA,1550-12-31
986540,STI,1635-12-31
892130,ANS,1671-12-31
...,...,...
1032596,KBO,2021-04-21
1032590,KBO,2021-04-21
1032572,KBO,2021-04-21
1035689,KBO,2021-04-21
